# More complex topological structures - Clos network example

## Imports

In [1]:
import random

from ns import ns

Let's simulate a Clos network unsing NS-3 as described in the following figure:

![Clos network](fig/clos_pure.drawio.png)

## Definition of the structure of Clos network

In [2]:
NODES = (
    4,  # input nodes
    2,  # input switches
    3,  # middle switches
    2,  # output switches
    4,  # output nodes
)


# Check the symmetry of the network and appropriate length
assert len(NODES) == 5
assert NODES[0] % NODES[1] == 0
assert NODES[-1] % NODES[-2] == 0
assert NODES[0] == NODES[-1]
assert NODES[1] == NODES[-2]
assert NODES[0] == NODES[1] ** 2

## Supplementary functions

In [3]:
# Here, you can add functions that will help you simulate the network
from lib.utils import get_node_ip_from_idx

## Enable application-layer logging

In [4]:
ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.core.LOG_LEVEL_INFO)

## Create Nodes

In [5]:
# Create nodes for each stage
node_containers = [ns.network.NodeContainer() for _ in NODES]
x = 10.0
for num, c in zip(NODES, node_containers):
    y = 10.0
    c.Create(num)
    for node_idx in range(c.GetN()):
        node = c.Get(node_idx).__deref__()
        ns.AnimationInterface.SetConstantPosition(node, x, y)
        y += 10.0
    x += 10.0

## Install devices and link the layers together

In [6]:
p2p = ns.point_to_point.PointToPointHelper()
devices = {i: [] for i in range(len(NODES) - 1)}  # inter layer connections hence -1

# Connect input switches to middle switches
for connection_layer in devices.keys():
    if connection_layer == 0:
        for node_idx in range(NODES[0]):
            switch_idx = node_idx // NODES[1]
            devices[connection_layer].append(
                {
                    "connection_layer": connection_layer,
                    "l": node_idx,
                    "r": switch_idx,
                    "devices": p2p.Install(
                        node_containers[connection_layer].Get(node_idx),
                        node_containers[connection_layer + 1].Get(switch_idx),
                    ),  # returns NetDeviceContainer
                }
            )

    elif connection_layer == len(NODES) - 2:  # -2 for last layer
        # connections to sources/sinks with single device
        for node_idx in range(NODES[-1]):
            switch_idx = node_idx // NODES[-2]
            devices[connection_layer].append(
                {
                    "connection_layer": connection_layer,
                    "l": switch_idx,
                    "r": node_idx,
                    "devices": p2p.Install(
                        node_containers[connection_layer].Get(switch_idx),
                        node_containers[connection_layer + 1].Get(node_idx),
                    ),  # returns NetDeviceContainer
                }
            )
    else:
        # interswitch connections hence more than one device
        for i in range(NODES[connection_layer]):  # left side
            for j in range(NODES[connection_layer + 1]):  # right side
                devices[connection_layer].append(
                    {
                        "connection_layer": connection_layer,
                        "l": i,
                        "r": j,
                        "devices": p2p.Install(
                            node_containers[connection_layer].Get(i),
                            node_containers[connection_layer + 1].Get(j),
                        ),  # returns NetDeviceContainer
                    }
                )

## Add IP stack and addressing

In [7]:
# Add Internet stack
internet = ns.internet.InternetStackHelper()
internet.InstallAll()

# Assign IP addresses
address = ns.internet.Ipv4AddressHelper()
for layer, devs in devices.items():
    for idx, dev in enumerate(devs):
        base_ip = f"10.{layer}.{idx}.0"
        address.SetBase(
            ns.network.Ipv4Address(base_ip), ns.network.Ipv4Mask("255.255.255.0")
        )
        address_container = address.Assign(dev["devices"])
        # store ip addresses for future use
        dev["ip_addresses"] = (
            address_container.GetAddress(0),
            address_container.GetAddress(1),
        )

ns.internet.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

## Add applications

In [8]:
# Set up traffic
source_apps = ns.network.ApplicationContainer()
dest_apps = ns.network.ApplicationContainer()

# for i in range(num_switches_per_stage):

source_nodes_idxs = list(range(NODES[0]))
dest_nodes_idxs = list(range(NODES[-1]))
for i in range(NODES[0]):
    source_node_idx = random.choice(source_nodes_idxs)
    dest_node_idx = random.choice(dest_nodes_idxs)
    source_nodes_idxs.remove(source_node_idx)
    dest_nodes_idxs.remove(dest_node_idx)

    source_node = node_containers[0].Get(source_node_idx)
    dest_node = node_containers[-1].Get(dest_node_idx)

    source_addr = get_node_ip_from_idx(devices, 0, source_node_idx)
    dest_addr = get_node_ip_from_idx(
        devices, len(NODES) - 2, right_idx=dest_node_idx
    )
    print(source_addr, dest_addr)

    port = 9

    echo_srv_helper = ns.UdpEchoServerHelper(port)
    dest_apps.Add(echo_srv_helper.Install(dest_node))

    echo_client_helper = ns.applications.UdpEchoClientHelper(dest_addr.ConvertTo(), port)
    echo_client_helper.SetAttribute("MaxPackets", ns.core.UintegerValue(10))
    # echo_client.SetAttribute("Interval", ns.core.TimeValue(ns.core.Seconds(interval)))
    # echo_client.SetAttribute("PacketSize", ns.core.UintegerValue(packet_size))
    source_apps.Add(echo_client_helper.Install(source_node))

10.0.2.1 10.3.1.2
10.0.3.1 10.3.3.2
10.0.0.1 10.3.2.2
10.0.1.1 10.3.0.2


## Enable PCAP logging

In [9]:
p2p.EnablePcap("xxxxx", devices[0][0]['devices'].Get(1), True)

## Enable Ascii logs

In [10]:
p2p.EnableAsciiAll("clos-ascii-trace.tr")

## Enable simulation logging

In [11]:
animator = ns.netanim.AnimationInterface("clos-animation.xml")

## Time the simulation

In [12]:
source_apps.Start(ns.core.Seconds(2.0))
source_apps.Stop(ns.core.Seconds(10.0))
dest_apps.Start(ns.core.Seconds(1.0))
dest_apps.Stop(ns.core.Seconds(10.0))

ns.Simulator.Stop(ns.Seconds(10))

## Run the simulation

In [13]:
ns.core.Simulator.Run()
ns.core.Simulator.Destroy()

At time +2s client sent 100 bytes to 10.3.2.2 port 9
At time +2s client sent 100 bytes to 10.3.0.2 port 9
At time +2s client sent 100 bytes to 10.3.1.2 port 9
At time +2s client sent 100 bytes to 10.3.3.2 port 9
At time +2.12695s server received 100 bytes from 10.0.0.1 port 49153
At time +2.12695s server sent 100 bytes to 10.0.0.1 port 49153
At time +2.12695s server received 100 bytes from 10.0.2.1 port 49153
At time +2.12695s server sent 100 bytes to 10.0.2.1 port 49153
At time +2.15869s server received 100 bytes from 10.0.3.1 port 49153
At time +2.15869s server sent 100 bytes to 10.0.3.1 port 49153
At time +2.15869s server received 100 bytes from 10.0.1.1 port 49153
At time +2.15869s server sent 100 bytes to 10.0.1.1 port 49153
At time +2.25391s client received 100 bytes from 10.3.2.2 port 9
At time +2.25391s client received 100 bytes from 10.3.1.2 port 9
At time +2.28564s client received 100 bytes from 10.3.0.2 port 9
At time +2.28564s client received 100 bytes from 10.3.3.2 port 9


## Task

1. Limit the network bandwidth and try to check a non-blocking nature of Clos network.